## Sport predictor using Olympics Data

Using an Olympics data set from: https://www.kaggle.com/datasets/heesoo37/120-years-of-olympic-history-athletes-and-results?resource=download, this project seeks to use Age, sex, height, weight and Team to predict what sport they should be competing in. 

#### CSV file columns: 
Name - Athlete's name
Sex - M or F
Age - Integer
Height - In centimeters
Weight - In kilograms
Team - Team name
NOC - National Olympic Committee 3-letter code
Games - Year and season
Year - Integer
Season - Summer or Winter
City - Host city
Sport - Sport
Event - Event
Medal - Gold, Silver, Bronze, or NAN

#### Imports 

In [64]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from patsy import dmatrices

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.tree import export_graphviz

import graphviz
from graphviz import Source

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Part 1: Cleaning and Discovering the data 


In [65]:
df = pd.read_csv('athlete_events.csv')
regions = pd.read_csv('noc_regions.csv')
df.shape

(271116, 15)

#### Check the values of the columns 

In [66]:
column_list =  df[["Name","Sex","Team", "NOC", "Games", "Season","City","Sport", "Event","Medal","Age","Height","Weight", "ID", "Year"]].columns

for i in column_list:
    #print(df.[i].unique())
    df2 = pd.unique(df[[i]].values.ravel('k'))
    print('Unique values from column '+ i + "\n", df2)

Unique values from column Name
 ['A Dijiang' 'A Lamusi' 'Gunnar Nielsen Aaby' ... 'Andrzej ya' 'Piotr ya'
 'Tomasz Ireneusz ya']
Unique values from column Sex
 ['M' 'F']
Unique values from column Team
 ['China' 'Denmark' 'Denmark/Sweden' ... 'Solos Carex' 'Dow Jones' 'Digby']
Unique values from column NOC
 ['CHN' 'DEN' 'NED' 'USA' 'FIN' 'NOR' 'ROU' 'EST' 'FRA' 'MAR' 'ESP' 'EGY'
 'IRI' 'BUL' 'ITA' 'CHA' 'AZE' 'SUD' 'RUS' 'ARG' 'CUB' 'BLR' 'GRE' 'CMR'
 'TUR' 'CHI' 'MEX' 'URS' 'NCA' 'HUN' 'NGR' 'ALG' 'KUW' 'BRN' 'PAK' 'IRQ'
 'UAR' 'LIB' 'QAT' 'MAS' 'GER' 'CAN' 'IRL' 'AUS' 'RSA' 'ERI' 'TAN' 'JOR'
 'TUN' 'LBA' 'BEL' 'DJI' 'PLE' 'COM' 'KAZ' 'BRU' 'IND' 'KSA' 'SYR' 'MDV'
 'ETH' 'UAE' 'YAR' 'INA' 'PHI' 'SGP' 'UZB' 'KGZ' 'TJK' 'EUN' 'JPN' 'CGO'
 'SUI' 'BRA' 'FRG' 'GDR' 'MON' 'ISR' 'URU' 'SWE' 'ISV' 'SRI' 'ARM' 'CIV'
 'KEN' 'BEN' 'UKR' 'GBR' 'GHA' 'SOM' 'LAT' 'NIG' 'MLI' 'AFG' 'POL' 'CRC'
 'PAN' 'GEO' 'SLO' 'CRO' 'GUY' 'NZL' 'POR' 'PAR' 'ANG' 'VEN' 'COL' 'BAN'
 'PER' 'ESA' 'PUR' 'UGA' 'HON' 'ECU

#### Check and Deal with NaN values 

In [67]:
df.isnull().values.any()

True

In [68]:
#### Find all NaN values and replace with 0 
df = df.fillna(0)

In [69]:
df.isnull().values.any()

False

#### Types of Data 

In [70]:
df.dtypes

ID          int64
Name       object
Sex        object
Age       float64
Height    float64
Weight    float64
Team       object
NOC        object
Games      object
Year        int64
Season     object
City       object
Sport      object
Event      object
Medal      object
dtype: object

In [71]:
categorical_columns = df[["Name","Sex","Team", "NOC", "Games", "Season","City","Sport", "Event","Medal"]].columns

numerical_columns = df[["Age","Height","Weight" ]].columns

for column in categorical_columns:
    df[column] = df[column].astype('category')

for i in numerical_columns:
    df[i] = df[i].astype('int')




#### Create a new column for possible use later

In [72]:

conditions = [
    (df['Medal'] == 'Bronze'),
    (df['Medal'] == "Silver"),
    (df['Medal'] == 'Gold'),
    (df['Medal'].isna()),
    ]

#values = ['Winner3', 'Winner2', 'Winner1','Not qualified']
values2 = ['True', 'True', 'True', "False"]


In [73]:
#df['Status'] = np.select(conditions, values)
df['Winner'] = np.select(conditions, values2)
# display updated DataFrame
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Winner
0,1,A Dijiang,M,24,180,80,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,0,0
1,2,A Lamusi,M,23,170,60,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,0,0
2,3,Gunnar Nielsen Aaby,M,24,0,0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,0,0
3,4,Edgar Lindenau Aabye,M,34,0,0,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,True
4,5,Christine Jacoba Aaftink,F,21,185,82,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,0,0


In [74]:
#There are duplicates that need to be dealt with 
print('Number of duplicate rows in the table is: ', df.duplicated().sum())
print('Number of duplicate  columns in the table is: ', df.columns.duplicated().sum())

Number of duplicate rows in the table is:  1385
Number of duplicate  columns in the table is:  0


In [75]:
['CHN' 'DEN' 'NED' 'USA' 'FIN' 'NOR' 'ROU' 'EST' 'FRA' 'MAR' 'ESP' 'EGY'
 'IRI' 'BUL' 'ITA' 'CHA' 'AZE' 'SUD' 'RUS' 'ARG' 'CUB' 'BLR' 'GRE' 'CMR'
 'TUR' 'CHI' 'MEX' 'URS' 'NCA' 'HUN' 'NGR' 'ALG' 'KUW' 'BRN' 'PAK' 'IRQ'
 'UAR' 'LIB' 'QAT' 'MAS' 'GER' 'CAN' 'IRL' 'AUS' 'RSA' 'ERI' 'TAN' 'JOR'
 'TUN' 'LBA' 'BEL' 'DJI' 'PLE' 'COM' 'KAZ' 'BRU' 'IND' 'KSA' 'SYR' 'MDV'
 'ETH' 'UAE' 'YAR' 'INA' 'PHI' 'SGP' 'UZB' 'KGZ' 'TJK' 'EUN' 'JPN' 'CGO'
 'SUI' 'BRA' 'FRG' 'GDR' 'MON' 'ISR' 'URU' 'SWE' 'ISV' 'SRI' 'ARM' 'CIV'
 'KEN' 'BEN' 'UKR' 'GBR' 'GHA' 'SOM' 'LAT' 'NIG' 'MLI' 'AFG' 'POL' 'CRC'
 'PAN' 'GEO' 'SLO' 'CRO' 'GUY' 'NZL' 'POR' 'PAR' 'ANG' 'VEN' 'COL' 'BAN'
 'PER' 'ESA' 'PUR' 'UGA' 'HON' 'ECU' 'TKM' 'MRI' 'SEY' 'TCH' 'LUX' 'MTN'
 'CZE' 'SKN' 'TTO' 'DOM' 'VIN' 'JAM' 'LBR' 'SUR' 'NEP' 'MGL' 'AUT' 'PLW'
 'LTU' 'TOG' 'NAM' 'AHO' 'ISL' 'ASA' 'SAM' 'RWA' 'DMA' 'HAI' 'MLT' 'CYP'
 'GUI' 'BIZ' 'YMD' 'KOR' 'THA' 'BER' 'ANZ' 'SCG' 'SLE' 'PNG' 'YEM' 'IOA'
 'OMA' 'FIJ' 'VAN' 'MDA' 'YUG' 'BAH' 'GUA' 'SRB' 'IVB' 'MOZ' 'CAF' 'MAD'
 'MAL' 'BIH' 'GUM' 'CAY' 'SVK' 'BAR' 'GBS' 'TLS' 'COD' 'GAB' 'SMR' 'LAO'
 'BOT' 'ROT' 'CAM' 'PRK' 'SOL' 'SEN' 'CPV' 'CRT' 'GEQ' 'BOL' 'SAA' 'AND'
 'ANT' 'ZIM' 'GRN' 'HKG' 'LCA' 'FSM' 'MYA' 'MAW' 'ZAM' 'RHO' 'TPE' 'STP'
 'MKD' 'BOH' 'TGA' 'LIE' 'MNE' 'GAM' 'COK' 'ALB' 'WIF' 'SWZ' 'BUR' 'NBO'
 'BDI' 'ARU' 'NRU' 'VNM' 'VIE' 'BHU' 'MHL' 'KIR' 'UNK' 'TUV' 'NFL' 'KOS'
 'SSD' 'LES']

['CHNDENNEDUSAFINNORROUESTFRAMARESPEGYIRIBULITACHAAZESUDRUSARGCUBBLRGRECMRTURCHIMEXURSNCAHUNNGRALGKUWBRNPAKIRQUARLIBQATMASGERCANIRLAUSRSAERITANJORTUNLBABELDJIPLECOMKAZBRUINDKSASYRMDVETHUAEYARINAPHISGPUZBKGZTJKEUNJPNCGOSUIBRAFRGGDRMONISRURUSWEISVSRIARMCIVKENBENUKRGBRGHASOMLATNIGMLIAFGPOLCRCPANGEOSLOCROGUYNZLPORPARANGVENCOLBANPERESAPURUGAHONECUTKMMRISEYTCHLUXMTNCZESKNTTODOMVINJAMLBRSURNEPMGLAUTPLWLTUTOGNAMAHOISLASASAMRWADMAHAIMLTCYPGUIBIZYMDKORTHABERANZSCGSLEPNGYEMIOAOMAFIJVANMDAYUGBAHGUASRBIVBMOZCAFMADMALBIHGUMCAYSVKBARGBSTLSCODGABSMRLAOBOTROTCAMPRKSOLSENCPVCRTGEQBOLSAAANDANTZIMGRNHKGLCAFSMMYAMAWZAMRHOTPESTPMKDBOHTGALIEMNEGAMCOKALBWIFSWZBURNBOBDIARUNRUVNMVIEBHUMHLKIRUNKTUVNFLKOSSSDLES']

## Part 2: Prepare for Machine Learning 

#### Drop columns
The aim of this project is to take someones Sex, Age, Height, Weight and Nationality (Team) and train and ML model to find someone's sport and later possibly their likely hood of winning or medal. 

In [76]:
newdf = df.drop(columns=['ID','Name', 'Team', 'Year','Season','City','Event','Medal',"Games", "Winner"])
newdf.head(5)

,Sex,Age,Height,Weight,NOC,Sport
0,M,24,180,80,CHN,Basketball
1,M,23,170,60,CHN,Judo
2,M,24,0,0,DEN,Football
3,M,34,0,0,DEN,Tug-Of-War
4,F,21,185,82,NED,Speed Skating


In [77]:
#Change Sex to 0 and 1 
newdf['Sex'] = df['Sex'].map({'M': 1, 'F': 0})
newdf.head(5)

,Sex,Age,Height,Weight,NOC,Sport
0,1,24,180,80,CHN,Basketball
1,1,23,170,60,CHN,Judo
2,1,24,0,0,DEN,Football
3,1,34,0,0,DEN,Tug-Of-War
4,0,21,185,82,NED,Speed Skating


#### Seprate target into y 

In [100]:
#### 
#newdf = newdf.sample(frac = 1) shuffled - laer use 

# The target y is sport 
#ydf = newdf["Sport"]
#ydf = ydf.rename("SportColumnName")
ydf = pd.DataFrame({"Sports": newdf["Sport"]})

# X is all the everything else 
Xdf = newdf.drop(["Sport"], axis=1)




### Encode the categorical Data 

In [104]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output = False).set_output(transform='pandas')
ohetransform =ohe.fit_transform(ydf[["Sports"]])
y = pd.concat([ydf,ohetransform], axis=1).drop(columns = ["Sports"])


,Sports_Aeronautics,Sports_Alpine Skiing,Sports_Alpinism,Sports_Archery,Sports_Art Competitions,Sports_Athletics,Sports_Badminton,Sports_Baseball,Sports_Basketball,Sports_Basque Pelota,...,Sports_Table Tennis,Sports_Taekwondo,Sports_Tennis,Sports_Trampolining,Sports_Triathlon,Sports_Tug-Of-War,Sports_Volleyball,Sports_Water Polo,Sports_Weightlifting,Sports_Wrestling
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
271112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
271113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
271114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output = False).set_output(transform='pandas')
ohetransform =ohe.fit_transform(Xdf[["NOC"]])
X = pd.concat([Xdf,ohetransform], axis=1).drop(columns = ["NOC"])




## Part 3: Machine learning Models

In [105]:
## Split the dataset into two datasets: 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100, )

In [108]:
# need to reset the index to allow contatenation with predicted values otherwise not joining on same index...
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)


In [109]:
multiple_linreg = LinearRegression().fit(X_train, y_train)


DecisionTreeClassifier()

In [36]:
lr = LinearRegression()
lr.fit(X_train, y_train) 

ValueError: could not convert string to float: 'Ice Hockey'

In [115]:
print("\nDescriptive features in X:\n", X_train.head(5))
print("\nTarget feature in y:\n", y_train.head(5))


Descriptive features in X:
   Sex  Age  Height  Weight  NOC_AFG  NOC_AHO  NOC_ALB  NOC_ALG  NOC_AND  \
0   1   28     181      88      0.0      0.0      0.0      0.0      0.0   
1   0   19     175      70      0.0      0.0      0.0      0.0      0.0   
2   1   23     184      76      0.0      0.0      0.0      0.0      0.0   
3   1   26     195     102      0.0      0.0      0.0      0.0      0.0   
4   1   39       0       0      0.0      0.0      0.0      0.0      0.0   

   NOC_ANG  ...  NOC_VIE  NOC_VIN  NOC_VNM  NOC_WIF  NOC_YAR  NOC_YEM  \
0      0.0  ...      0.0      0.0      0.0      0.0      0.0      0.0   
1      0.0  ...      0.0      0.0      0.0      0.0      0.0      0.0   
2      0.0  ...      0.0      0.0      0.0      0.0      0.0      0.0   
3      0.0  ...      0.0      0.0      0.0      0.0      0.0      0.0   
4      0.0  ...      0.0      0.0      0.0      0.0      0.0      0.0   

   NOC_YMD  NOC_YUG  NOC_ZAM  NOC_ZIM  
0      0.0      0.0      0.0      0.0  
1